In [1]:
#Главные задачи, которые должен выполнять скрипт:
#
#Извлекать реплики с приветствием – где менеджер поздоровался. 
#Извлекать реплики, где менеджер представил себя. 
#Извлекать имя менеджера. 
#Извлекать название компании. 
#Извлекать реплики, где менеджер попрощался.
#Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


In [2]:
# Я подготовил список приветствий из базы данных викисловаря. Я не стал брать неформальные, а также не стал брать 
# "Алло" так как это, как мне кажется неуместное приветствие для менеджера.

In [3]:
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer 

In [4]:
df = pd.read_csv("./test_data.csv", sep=",")
stemmer = SnowballStemmer("russian") 

In [5]:
df_extracted = pd.DataFrame(columns=['dialog_id','greeting', 'introduction', 'manager_name', 'company_name', 'farewell', 'was_polite'])

In [6]:
df.head(5)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [7]:
df_extracted

,dialog_id,greeting,introduction,manager_name,company_name,farewell,was_polite


In [8]:
class Dialog:
    def __init__(self, dlg_id):
        self.id = dlg_id
        self.greeted = False
        self.greeting = ""
        self.introduction = ""
        self.farewell = ""
        self.goodbyed = False
        self.name = ""
        self.company = ""
        self.was_polite = False
        
    def load_to_df(self):
        self.was_polite = self.greeted and self.goodbyed
        data = [self.id, self.greeting, self.introduction, self.name, self.company, self.farewell, self.was_polite]
        df_extracted.loc[len(df_extracted.index)] = data


In [9]:
def normalise(line):
    normalised = [stemmer.stem(t) for t in line]
    return normalised

In [10]:
def fetch(text: str, dialog: Dialog, line_n: int):
# Приветствие
    if not dialog.greeted and any(greeting in text for greeting in greetings):
        dialog.greeted = True
        dialog.greeting = text
        
# Прощание
    if any(goodbye in text for goodbye in goodbyes):
        dialog.goodbyed = True
        dialog.farewell = text
        
# Имя мэнеджера
    if (dialog.name != '' or line_n > 5):
        return dialog
    else:
        text_list = text.split(sep=" ")
        tokenized_text = normalise(text_list)   
        for i in range(len(tokenized_text) - 1):
            # Быстрый поиск по ключевым словам
            two_tokens = tokenized_text[i] + " " + tokenized_text[i + 1]    

            if any(intro in two_tokens for intro in intros): 
                if tokenized_text[i+1] in names:
                    dialog.name = text_list[i+1]
                    dialog.introduction = " ".join(text_list)
                    break
            # Кропотливый поиск, если быстрый не помог
            if text_list[i] in names:
                dialog.name = text_list[i]
                dialog.introduction = " ".join(text_list)
                break
# Компания        
    if (dialog.company != ''):
        return dialog
    else:
        text_list = text.split(sep=" ")
        tokenized_text = normalise(text_list)

        for i in range(len(tokenized_text)):
            if tokenized_text[i] == company_token:
                if i == len(tokenized_text) - 1:
                       dialog.company = text_list[i - 1]
                else:
                       dialog.company = text_list[i + 1]
                break
    return dialog

In [11]:
def process(line: pd.DataFrame, dialog: Dialog) -> Dialog:
    if line['role'] == 'client':
        return dialog
    
    if line['dlg_id'] != dialog.id:
        dialog.load_to_df()
        dialog = Dialog(line['dlg_id'])
        
           
    text = line['text']
    
    return fetch(text, dialog, line['line_n'])

In [12]:
goodbyes_raw = ["до свиданья",
"всего доброго",
"всего хорошего",
"удачного дня",
"до свидания",
"добрый день",
"добрый вечер",
"прощай",
"прощайте",
"до скорого",
"спокойной ночи",
"увидимся",
"счастливо"]

# Источник базы данных: https://ru.wiktionary.org/wiki/%D0%B7%D0%B4%D1%80%D0%B0%D0%B2%D1%81%D1%82%D0%B2%D1%83%D0%B9%D1%82%D0%B5

In [13]:
greetings_raw = ['здравствуйте', 'здравствуй', 'приветствую', 'привет', 'доброго времени суток', 'добрый день', 'доброе утро', 'добрый вечер']

# Источник базы данных: https://ru.wiktionary.org/wiki/%D0%B4%D0%BE_%D1%81%D0%B2%D0%B8%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F


In [14]:
intros = ['мен зовут', 'мое им', 'вам звон']
names_df =  pd.read_csv("./russian_names.csv", sep=';')
print(names_df.head())
names = names_df['Name'].tolist()
names = [name.lower() for name in names]

# Источник базы данных: https://mydata.biz/ru/catalog/databases/names_db

      ID      Name Sex  PeoplesCount     WhenPeoplesCount      Source
0  19903     Аалия   Ж            13  23.06.2016 13:39:41  myData.biz
1  19904  Аанжелла   Ж             0  23.06.2016 13:39:46  myData.biz
2  19905       Аба   Ж          1000  23.06.2016 13:39:55  myData.biz
3  19906      Абав   Ж             0  23.06.2016 13:40:02  myData.biz
4  19907      Абам   Ж            32  23.06.2016 13:40:11  myData.biz


In [15]:
company_token = stemmer.stem("компания")

greetings = normalise(greetings_raw)
goodbyes = normalise(goodbyes_raw)


In [16]:
dialog = Dialog(0)

for _, line in df.iterrows():
        dialog = process(line, dialog)


In [17]:
df_extracted.to_csv("analysis.csv")

In [18]:
df_extracted

,dialog_id,greeting,introduction,manager_name,company_name,farewell,was_polite
0,0,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал,Всего хорошего до свидания,True
1,1,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал,Угу да вижу я эту почту хорошо тогда исправлю ...,True
2,2,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал,,False
3,3,Алло дмитрий добрый день,Алло дмитрий добрый день,дмитрий,,Угу все хорошо да понедельника тогда всего доб...,True
4,4,,Вот по виду платежи пообщаться помните мы вот ...,вас,,Во вторник все ну с вами да тогда до вторника ...,False
